In [1]:
import onnx
from onnx2keras import onnx_to_keras

2022-08-24 10:23:32.896579: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model

In [3]:
def onnx2txt(model, txt_file):
#     onnx_model = onnx.load(onnx_file)

#     # load keras model from the h5 file
#     model = onnx_to_keras(onnx_model, ['input'])
    print("============model summary============")
    print(model.summary())

    # get the input dim
    input_dim = model.layers[0].get_config()['batch_input_shape'][-1]
    # get the output dim
    output_dim = model.layers[-1].get_config()['units']
    # var to record number of hidden neurons
    num_of_hidden_neurons = []
    # var to record types of activation function
    activations = []

    for _layer in model.layers[1:]:
        layer_config = _layer.get_config()
        if 'layers' in layer_config:
            # if there is a functional model in a layer, this layer actually
            # contains layers that cannot be read from the model directly.
            for _layer_in_model in _layer.layers[1:]:
                layer_config = _layer_in_model.get_config()
                # append the current layer's number of neurons
                num_of_hidden_neurons.append(layer_config['units'])
                # append the current layer's activation function type
                activations.append(layer_config['activation'])
        else:
            if 'units' in layer_config:
                # append the current layer's number of neurons
                num_of_hidden_neurons.append(layer_config['units'])
                # append the current layer's activation function type
                activations.append(layer_config['activation'])
            elif 'activation' in layer_config:
                # for layers that only have the activation function, rewrite
                # the latest activation function, probably linear, with current
                # layer's activation function type.
                activations[-1] = layer_config['activation']

    # get the number of hidden layers
    num_of_hidden_layer = len(activations) - 1

    with open(txt_file, 'w') as output_file:
        # write the neural network architecture
        output_file.write('{}'.format(input_dim) + '\n')
        output_file.write('{}'.format(output_dim) + '\n')
        output_file.write('{}'.format(num_of_hidden_layer) + '\n')

        for _num_neurons in num_of_hidden_neurons[:-1]:
            output_file.write('{}'.format(_num_neurons) + '\n')

        for _activation in activations:
            if _activation == 'linear':
                _activation = 'Affine'
            output_file.write('{}'.format(_activation) + '\n')

        # write weights and biases
        for _layer in model.layers[1:]:
            layer_config = _layer.get_config()
            if 'layers' in layer_config:
                # if there is a functional model in a layer, this layer
                # actually contains layers that cannot be read from the model
                # directly.
                for _layer_in_model in _layer.layers[1:]:
                    weights, biases = _layer_in_model.get_weights()
                    # wrtie weights
                    for _col in range(weights.shape[1]):
                        for _row in range(weights.shape[0]):
                            output_file.write(
                                '{}'.format(weights[_row, _col]) + '\n'
                            )
                    # write biases
                    for _idx_neuron in range(biases.shape[0]):
                        output_file.write('{}'.format(
                            biases[_idx_neuron]) + '\n'
                        )

            else:
                if 'units' in layer_config:
                    weights, biases = _layer.get_weights()
                    # wrtie weights
                    for _col in range(weights.shape[1]):
                        for _row in range(weights.shape[0]):
                            output_file.write('{}'.format(
                                weights[_row, _col]) + '\n'
                            )
                    # write biases
                    for _idx_neuron in range(biases.shape[0]):
                        output_file.write('{}'.format(biases[_idx_neuron]) + '\n')

        # write default scalar and offset
        output_file.write('{}'.format(0) + '\n')
        output_file.write('{}'.format(1) + '\n')
    print("============{} saved============".format(txt_file))
    print("============Done============")

In [3]:
def keras2txt(model, txt_file):
    # load keras model from the h5 file
    # model = load_model(h5_file)
    print("============keras model summary============")
    print(model.summary())

    # get the input dim
    if txt_file == './Airplane/controller_airplane_POLAR':
        input_dim = 12
    elif txt_file == './Sherlock-B10/controllerB_POLAR' \
            or txt_file == './Sherlock-B9/controllerTora_POLAR' \
            or txt_file == './Double_Pendulum/controller_double_pendulum_less_robust_POLAR' \
            or txt_file == './Double_Pendulum/controller_double_pendulum_more_robust_POLAR':
        input_dim = 4
    elif txt_file == './Single_Pendulum/controller_single_pendulum_POLAR':
        input_dim = 2
    elif 'VCAS' in txt_file:
        input_dim = 3
    else:
        input_dim = model.layers[0].get_config()['batch_input_shape'][1]
    # get the output dim
    output_dim = model.layers[-1].get_config()['units']
    # var to record number of hidden neurons
    num_of_hidden_neurons = []
    # var to record types of activation function
    activations = []

    for _layer in model.layers:
        layer_config = _layer.get_config()
        if 'layers' in layer_config:
            # if there is a functional model in a layer, this layer actually
            # contains layers that cannot be read from the model directly.
            for _layer_in_model in _layer.layers[1:]:
                layer_config = _layer_in_model.get_config()
                # append the current layer's number of neurons
                num_of_hidden_neurons.append(layer_config['units'])
                # append the current layer's activation function type
                activations.append(layer_config['activation'])
        else:
            if 'units' in layer_config:
                # append the current layer's number of neurons
                num_of_hidden_neurons.append(layer_config['units'])
                # append the current layer's activation function type
                activations.append(layer_config['activation'])
            elif 'activation' in layer_config:
                # for layers that only have the activation function, rewrite
                # the latest activation function, probably linear, with current
                # layer's activation function type.
                activations[-1] = layer_config['activation']

    # get the number of hidden layers
    num_of_hidden_layer = len(activations) - 1

    with open(txt_file, 'w') as output_file:
        # write the neural network architecture
        output_file.write('{}'.format(input_dim) + '\n')
        output_file.write('{}'.format(output_dim) + '\n')
        output_file.write('{}'.format(num_of_hidden_layer) + '\n')

        for _num_neurons in num_of_hidden_neurons[:-1]:
            output_file.write('{}'.format(_num_neurons) + '\n')

        for _activation in activations:
            if _activation == 'linear':
                _activation = 'Affine'
            output_file.write('{}'.format(_activation) + '\n')

        # write weights and biases
        for _layer in model.layers:
            layer_config = _layer.get_config()
            if 'layers' in layer_config:
                # if there is a functional model in a layer, this layer
                # actually contains layers that cannot be read from the model
                # directly.
                for _layer_in_model in _layer.layers[1:]:
                    weights, biases = _layer_in_model.get_weights()
                    # wrtie weights
                    for _col in range(weights.shape[1]):
                        for _row in range(weights.shape[0]):
                            output_file.write(
                                '{}'.format(weights[_row, _col]) + '\n'
                            )
                    # write biases
                    for _idx_neuron in range(biases.shape[0]):
                        output_file.write('{}'.format(
                            biases[_idx_neuron]) + '\n'
                        )

            else:
                if 'units' in layer_config:
                    weights, biases = _layer.get_weights()
                    # wrtie weights
                    for _col in range(weights.shape[1]):
                        for _row in range(weights.shape[0]):
                            output_file.write('{}'.format(
                                weights[_row, _col]) + '\n'
                            )
                    # write biases
                    for _idx_neuron in range(biases.shape[0]):
                        output_file.write('{}'.format(biases[_idx_neuron]) + '\n')

        # write default scalar and offset
        output_file.write('{}'.format(0) + '\n')
        output_file.write('{}'.format(1) + '\n')
    print("============{} saved============".format(txt_file))
    print("============Done============")

In [ ]:
def onnx2txt_withOnnxAndKerasModel(onnx_model, model, txt_file):
#     onnx_model = onnx.load(onnx_file)

#     # load keras model from the h5 file
#     model = onnx_to_keras(onnx_model, ['input'])
    print("============model summary============")
    print(model.summary())

    # get the input dim
    input_dim = model.layers[0].get_config()['batch_input_shape'][-1]
    # get the output dim
    output_dim = model.layers[-1].get_config()['units']
    # var to record number of hidden neurons
    num_of_hidden_neurons = []
    # var to record types of activation function
    activations = []

    for _layer in model.layers[1:]:
        layer_config = _layer.get_config()
        if 'layers' in layer_config:
            # if there is a functional model in a layer, this layer actually
            # contains layers that cannot be read from the model directly.
            for _layer_in_model in _layer.layers[1:]:
                layer_config = _layer_in_model.get_config()
                # append the current layer's number of neurons
                num_of_hidden_neurons.append(layer_config['units'])
                # append the current layer's activation function type
                activations.append(layer_config['activation'])
        else:
            if 'units' in layer_config:
                # append the current layer's number of neurons
                num_of_hidden_neurons.append(layer_config['units'])
                # append the current layer's activation function type
                activations.append(layer_config['activation'])
            elif 'activation' in layer_config:
                # for layers that only have the activation function, rewrite
                # the latest activation function, probably linear, with current
                # layer's activation function type.
                activations[-1] = layer_config['activation']

    # get the number of hidden layers
    num_of_hidden_layer = len(activations) - 1

    with open(txt_file, 'w') as output_file:
        # write the neural network architecture
        output_file.write('{}'.format(input_dim) + '\n')
        output_file.write('{}'.format(output_dim) + '\n')
        output_file.write('{}'.format(num_of_hidden_layer) + '\n')

        for _num_neurons in num_of_hidden_neurons[:-1]:
            output_file.write('{}'.format(_num_neurons) + '\n')

        for _activation in activations:
            if _activation == 'linear':
                _activation = 'Affine'
            output_file.write('{}'.format(_activation) + '\n')

        # write weights and biases
        for _layer in model.layers[1:]:
            layer_config = _layer.get_config()
            if 'layers' in layer_config:
                # if there is a functional model in a layer, this layer
                # actually contains layers that cannot be read from the model
                # directly.
                for _layer_in_model in _layer.layers[1:]:
                    weights, biases = _layer_in_model.get_weights()
                    # wrtie weights
                    for _col in range(weights.shape[1]):
                        for _row in range(weights.shape[0]):
                            output_file.write(
                                '{}'.format(weights[_row, _col]) + '\n'
                            )
                    # write biases
                    for _idx_neuron in range(biases.shape[0]):
                        output_file.write('{}'.format(
                            biases[_idx_neuron]) + '\n'
                        )

            else:
                if 'units' in layer_config:
                    weights, biases = _layer.get_weights()
                    # wrtie weights
                    for _col in range(weights.shape[1]):
                        for _row in range(weights.shape[0]):
                            output_file.write('{}'.format(
                                weights[_row, _col]) + '\n'
                            )
                    # write biases
                    for _idx_neuron in range(biases.shape[0]):
                        output_file.write('{}'.format(biases[_idx_neuron]) + '\n')

        # write default scalar and offset
        output_file.write('{}'.format(0) + '\n')
        output_file.write('{}'.format(1) + '\n')
    print("============{} saved============".format(txt_file))
    print("============Done============")

# ACC

In [5]:
onnx_model = onnx.load('./ACC/controller_5_20.onnx')

In [6]:
model = onnx_to_keras(onnx_model, ['input'])

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input_AvgImg.
DEBUG:onnx2keras:Input 1 -> Operation_1_W.
DEBUG:onnx2keras:Input 2 -> Operation_1_B.
DEBUG:onnx2keras:Input 3 -> Operation_2_W.
DEBUG:onnx2keras:Input 4 -> Operation_2_B.
DEBUG:onnx2keras:Input 5 -> Operation_3_W.
DEBUG:onnx2keras:Input 6 -> Operation_3_B.
DEBUG:onnx2keras:Input 7 -> Operation_4_W.
DEBUG:onnx2keras:Input 8 -> Operation_4_B.
DEBUG:onnx2keras:Input 9 -> Operation_5_W.
DEBUG:onnx2keras:Input 10 -> Operation_5_B.
DEBUG:onnx2keras:Input 11 -> linear_6_W.
DEBUG:onnx2keras:Input 12 -> linear_6_B.
DEBUG:onnx2keras:Input 13 -> input.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> linear_6.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight input_AvgImg with shape (1, 1, 1, 5).
DEBUG:onnx2keras:Found weight Operation_1_W with shape (20, 5).
DEBUG:onnx2keras:Found weight Op

DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Relu
DEBUG:onnx2keras:node_name: relu_1
DEBUG:onnx2keras:node_params: {'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name Operation_1).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 1, 1, 20), dtype=tf.float32, name=None), name='relu_1/Relu:0', description="created by layer 'relu_1'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Gemm
DEBUG:onnx2keras:node_name: Operation_2
DEBUG:onnx2keras:node_params: {'alpha': 1.0, 'beta': 1.0, 'broadcast': 1, 'transA': 0, 'transB': 1, 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name relu_1)

DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:gemm:Convert GEMM with bias.
DEBUG:onnx2keras:gemm:Transposing W matrix.
DEBUG:onnx2keras:gemm:Input units 20, output units 1.
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 1, 1, 1), dtype=tf.float32, name=None), name='linear_6/BiasAdd:0', description="created by layer 'linear_6'")


In [10]:
onnx2txt(model, './ACC/controller_5_20_POLAR')

============model summary============
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 1, 1, 5)]    0                                            
__________________________________________________________________________________________________
input_Sub_const2 (Lambda)       (1, 1, 1, 5)         0           input[0][0]                      
__________________________________________________________________________________________________
input_Sub (Subtract)            (None, 1, 1, 5)      0           input[0][0]                      
                                                                 input_Sub_const2[0][0]           
__________________________________________________________________________________________________
Operation_1 (Dense)             (None, 1, 1, 20)     120

# Airplane

In [13]:
model = load_model('./Airplane/controller_airplane.h5')

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               1300      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 20)                2020      
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 126       
Total params: 13,546
Trainable params: 13,546
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.layers[0].get_config()

{'name': 'dense_1',
 'trainable': True,
 'dtype': 'float32',
 'units': 100,
 'activation': 'relu',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'scale': 1.0,
   'mode': 'fan_avg',
   'distribution': 'uniform',
   'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

In [18]:
keras2txt(model, './Airplane/controller_airplane_POLAR')

============keras model summary============
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               1300      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 20)                2020      
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 126       
Total params: 13,546
Trainable params: 13,546
Non-trainable params: 0
_________________________________________________________________
None
============./Airplane/controller_airplane_POLAR saved============
============Done============


# Sherlock-B10

In [25]:
model = load_model('./Sherlock-B10/controllerB_nnv.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 500)               2500      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1002      
Total params: 3,502
Trainable params: 3,502
Non-trainable params: 0
_________________________________________________________________


In [28]:
keras2txt(model, './Sherlock-B10/controllerB_POLAR')

============keras model summary============
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 500)               2500      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1002      
Total params: 3,502
Trainable params: 3,502
Non-trainable params: 0
_________________________________________________________________
None
============./Sherlock-B10/controllerB_POLAR saved============
============Done============


# Sherlock-B9

In [5]:
model = load_model('./Sherlock-B9/controllerTora_nnv.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               500       
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 20,801
Trainable params: 20,801
Non-trainable params: 0
_________________________________________________________________


2022-08-23 14:27:37.323250: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-08-23 14:27:37.357313: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2022-08-23 14:27:37.357986: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:06:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-08-23 14:27:37.358692: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID: 0000:09:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2

In [7]:
keras2txt(model, './Sherlock-B9/controllerTora_POLAR')

============keras model summary============
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               500       
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 20,801
Trainable params: 20,801
Non-trainable params: 0
_________________________________________________________________
None
============./Sherlock-B9/controllerTora_POLAR saved============
============Done============


# Docking

In [12]:
onnx_model = onnx.load('./Docking/bias_model.onnx')
model = onnx_to_keras(onnx_model, ['input'])
model.summary()

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> linear_5_Flatten.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight input_Mean with shape (1, 1, 4).
DEBUG:onnx2keras:Found weight linear_1_W with shape (4, 1, 1, 4).
DEBUG:onnx2keras:Found weight linear_1_B with shape (4,).
DEBUG:onnx2keras:Found weight linear_2_W with shape (256, 4, 1, 1).
DEBUG:onnx2keras:Found weight linear_2_B with shape (256,).
DEBUG:onnx2keras:Found weight Operation_3_W with shape (256, 256, 1, 1).
DEBUG:onnx2keras:Found weight Operation_3_B with shape (256,).
DEBUG:onnx2keras:Found weight linear_4_W with shape (4, 256, 1, 1).
DEBUG:onnx2keras:Found weight linear_4_B with shape (4,).
DEBUG:onnx2keras:Found weight linear_5_W with shape (4, 4, 1, 1).
DEBUG:onnx2keras:Found weight linear_5_B with shape (4,).
DEBUG:onnx2

DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name linear_5).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:flatten:Convert inputs to Keras/TF layers if needed.
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 4), dtype=tf.float32, name=None), name='linear_5_Flatten/Reshape:0', description="created by layer 'linear_5_Flatten'")


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 1, 1, 4)]    0                                            
__________________________________________________________________________________________________
input_Sub_const2 (Lambda)       (1, 1, 4)            0           input[0][0]                      
__________________________________________________________________________________________________
input_Sub (Subtract)            (None, 1, 1, 4)      0           input[0][0]                      
                                                                 input_Sub_const2[0][0]           
__________________________________________________________________________________________________
linear_1 (Conv2D)               (None, 4, 1, 1)      20          input_Sub[0][0]              

In [14]:
onnx_modelweights = onnx_model.graph.initializer

In [26]:
len(onnx_modelweights)

11

In [36]:
onnx.numpy_helper.to_array(onnx_modelweights[1])

array([[[[0.01, 0.  , 0.  , 0.  ]]],


       [[[0.  , 0.01, 0.  , 0.  ]]],


       [[[0.  , 0.  , 2.  , 0.  ]]],


       [[[0.  , 0.  , 0.  , 2.  ]]]], dtype=float32)

In [ ]:
onnx2txt(model, './Docking/bias_model_POLAR')

# Double Pendulum

In [4]:
model = load_model('./Double_Pendulum/controller_double_pendulum_less_robust.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 25)                125       
_________________________________________________________________
dense_2 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 52        
Total params: 827
Trainable params: 827
Non-trainable params: 0
_________________________________________________________________


2022-08-24 09:55:43.538622: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-08-24 09:55:43.573334: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2022-08-24 09:55:43.574344: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:06:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-08-24 09:55:43.575122: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID: 0000:09:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2

In [6]:
keras2txt(model, './Double_Pendulum/controller_double_pendulum_less_robust_POLAR')

============keras model summary============
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 25)                125       
_________________________________________________________________
dense_2 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 52        
Total params: 827
Trainable params: 827
Non-trainable params: 0
_________________________________________________________________
None
============./Double_Pendulum/controller_double_pendulum_less_robust_POLAR saved============
============Done============


In [7]:
model = load_model('./Double_Pendulum/controller_double_pendulum_more_robust.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 25)                125       
_________________________________________________________________
dense_2 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 52        
Total params: 827
Trainable params: 827
Non-trainable params: 0
_________________________________________________________________


In [9]:
keras2txt(model, './Double_Pendulum/controller_double_pendulum_more_robust_POLAR')

============keras model summary============
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 25)                125       
_________________________________________________________________
dense_2 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 52        
Total params: 827
Trainable params: 827
Non-trainable params: 0
_________________________________________________________________
None
============./Double_Pendulum/controller_double_pendulum_more_robust_POLAR saved============
============Done============


# Single Pendulum

In [10]:
model = load_model('./Single_Pendulum/controller_single_pendulum.h5')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 25)                75        
_________________________________________________________________
dense_5 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 26        
Total params: 751
Trainable params: 751
Non-trainable params: 0
_________________________________________________________________


In [13]:
keras2txt(model, './Single_Pendulum/controller_single_pendulum_POLAR')

============keras model summary============
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 25)                75        
_________________________________________________________________
dense_5 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 26        
Total params: 751
Trainable params: 751
Non-trainable params: 0
_________________________________________________________________
None
============./Single_Pendulum/controller_single_pendulum_POLAR saved============
============Done============


# VCAS

In [4]:
model = load_model('./VCAS/h5_networks/VertCAS_noResp_pra01_v9_20HU_200.nnet.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                80        
_________________________________________________________________
activation_1 (Activation)    (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                420       
_________________________________________________________________
activation_2 (Activation)    (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                420       
_________________________________________________________________
activation_3 (Activation)    (None, 20)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 20)               

2022-08-24 10:23:41.816049: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-08-24 10:23:41.867907: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2022-08-24 10:23:41.869262: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:06:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-08-24 10:23:41.870526: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID: 0000:09:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2

In [6]:
keras2txt(model, './VCAS/POLAR_networks/VertCAS_noResp_pra01_v9_20HU_200_POLAR')

============keras model summary============
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                80        
_________________________________________________________________
activation_1 (Activation)    (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                420       
_________________________________________________________________
activation_2 (Activation)    (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                420       
_________________________________________________________________
activation_3 (Activation)    (None, 20)                0         
_________________________________________________________________
dense_4 (D

In [8]:
from os import listdir
from os.path import isfile, join

h5_path = './VCAS/h5_networks/'
h5_files = [f for f in listdir(h5_path) if isfile(join(h5_path, f))]

In [11]:
for h5_file in h5_files:
    model = load_model('./VCAS/h5_networks/' + h5_file)
    keras2txt(model, './VCAS/POLAR_networks/' + h5_file[:-8] + '_POLAR')

============keras model summary============
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                80        
_________________________________________________________________
activation_1 (Activation)    (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                420       
_________________________________________________________________
activation_2 (Activation)    (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                420       
_________________________________________________________________
activation_3 (Activation)    (None, 20)                0         
_________________________________________________________________
dense_4 (D

============keras model summary============
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 20)                80        
_________________________________________________________________
activation_21 (Activation)   (None, 20)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 20)                420       
_________________________________________________________________
activation_22 (Activation)   (None, 20)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 20)                420       
_________________________________________________________________
activation_23 (Activation)   (None, 20)                0         
_________________________________________________________________
dense_28 (

============keras model summary============
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_49 (Dense)             (None, 20)                80        
_________________________________________________________________
activation_41 (Activation)   (None, 20)                0         
_________________________________________________________________
dense_50 (Dense)             (None, 20)                420       
_________________________________________________________________
activation_42 (Activation)   (None, 20)                0         
_________________________________________________________________
dense_51 (Dense)             (None, 20)                420       
_________________________________________________________________
activation_43 (Activation)   (None, 20)                0         
_________________________________________________________________
dense_52 (